## IEMS 394 - Biofuels Optimisation Model

Running list of assumptions:
- Some county name recurr within the set of states we have selected. Such counties have had their respective 2-letter state code appended to their name. They are enumerated below.
    1. Orange - CA,TX 
    2. Cass - MN, TX
    3. Lake - CA, MN
    4. Trinty - CA, TX
    5. Houston - MN, TX
    6. Polk - MN,TX
    7. Brown - MN,TX
    8. Clay - MN, TX
    9. Jackson - TX,MN
    10. Washington - MN, TX
    11. Martin - MN,TX
    
- future proofing changes:
    - Decision variable `n` assumes that vehicle types use a single type of fuel (it's not written in at all)

In [162]:
from pulp import *
from itertools import product
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from tqdm import tqdm

#### Sets

In [93]:
%%time
#import files from github

V = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/V.csv') #vehicle types
F = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/F.csv') #fuel types
E = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/E.csv') #driving env
R = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/R.csv') #counties
M = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/M.csv') #charging stations
S = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/S.csv') #states

F['fuel_type'] = F['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))

CPU times: user 86 ms, sys: 13.3 ms, total: 99.2 ms
Wall time: 2.17 s


In [94]:
VEHICLE_TYPES = list(V['vehicle_type'])
FUEL_TYPES = list(F['fuel_type'])
DRIVING_ENV = list(E['driving_environment'])
COUNTIES = list(R['county'])
CHARGING_STATIONS = list(M['filling_stations'])
STATES = list(S['state'])

#### Parameters
Parameters describe objects statically, and is constant in a single simulation. Parameters are only changed to adjust model behaviour.

In [66]:
%%time
#EF(f,s): Emission factor for fuel type f in state s, in gallons/mile  
EF = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/EF(f%2Cs).csv')
EF['fuel_type'] = EF['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#FE(v,f):Average fuel economy for vehicle type v using fuel f
FE = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/FE(v%2Cf).csv')
FE['fuel_type'] = FE['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#C(f): Cost of fuel type f 
C = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/C(F).csv')
#CC (v,s): Capital cost of vehicle type v in state s
CC = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/CC(v%2Cs).csv') 
#CG: cost of fuel/gallon
CG = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/CG(F).csv')
CG['fuel_type'] = CG['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#D: Emission decrease goals per year
D = 0.25
#W(s):Current yearly GHG emissions per state
W = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/W(s).csv')
#TM (v, f, r): Total miles for vehicle v using fuel f in county r
TM = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/TM(f%2Cs).csv')
# # TM.drop(['household_income_ID'],axis=1,inplace=True)
#N(r): Average income per county  
N = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/N(r).csv')
# N.drop(['household_income_ID'],axis=1,inplace=True)
#B(r) county + state linking table
B = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/B(r).csv')
#CF(v,f): Fuel consumption for vehicle type v using fuel f (1/fuel economy)
CF = FE
CF['fuel_economy'] = (1 / CF['fuel_economy'])

CPU times: user 132 ms, sys: 14 ms, total: 146 ms
Wall time: 1.55 s


In [4]:
v = V['vehicle_type']
f = F['fuel_type']
r = R['county']

#### Variables
Variables represent a model state and may change during simulation.

In [5]:
def variable_n():
    '''
    n(r,v,f) total optimal count of vehicle v using fuel type f in county r
    '''
    n = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    n['count'] = 0
    
    bev_elec = n[(n['vehicle_type']== 'BEV') & (n['fuel_type']== 'Electricity')]
    gas_e10 = n[(n['vehicle_type']== 'SIDI_ICE') & (n['fuel_type']== 'E10')]
    ffv_e85 = n[(n['vehicle_type']== 'FFV') & (n['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

In [6]:
def variable_fc():
    '''
    fc(r,v,f) Total fuel consumption by vehicle v using fuel type f in county r
    '''
    fc = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    fc['fuel_consumption'] = 0
    
    bev_elec = fc[(fc['vehicle_type']== 'BEV') & (fc['fuel_type']== 'Electricity')]
    gas_e10 = fc[(fc['vehicle_type']== 'SIDI_ICE') & (fc['fuel_type']== 'E10')]
    ffv_e85 = fc[(fc['vehicle_type']== 'FFV') & (fc['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

In [7]:
def variable_oc():
    '''
    oc(r,v,f) Operating cost per mile for vehicle v using fuel type f in county r 
    '''
    oc = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    oc['operating_cost'] = 0
    
    bev_elec = oc[(oc['vehicle_type']== 'BEV') & (oc['fuel_type']== 'Electricity')]
    gas_e10 = oc[(oc['vehicle_type']== 'SIDI_ICE') & (oc['fuel_type']== 'E10')]
    ffv_e85 = oc[(oc['vehicle_type']== 'FFV') & (oc['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result 

In [8]:
def variable_tac():
    '''
    tac(r,v,f) Total annual cost of vehicle v using fuel type f in county r 
    '''
    tac = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    tac['total_annual_vehicle_cost'] = 0
    
    bev_elec = tac[(tac['vehicle_type']== 'BEV') & (tac['fuel_type']== 'Electricity')]
    gas_e10 = tac[(tac['vehicle_type']== 'SIDI_ICE') & (tac['fuel_type']== 'E10')]
    ffv_e85 = tac[(tac['vehicle_type']== 'FFV') & (tac['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

In [27]:
def variable_ce():
    '''
    ce(r,v,f)  GHG emission per year of vehicle v using fuel type f in county r 
    '''
    ce = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    ce['emission_per_year'] = 0
    
    bev_elec = ce[(ce['vehicle_type']== 'BEV') & (ce['fuel_type']== 'Electricity')]
    gas_e10 = ce[(ce['vehicle_type']== 'SIDI_ICE') & (ce['fuel_type']== 'E10')]
    ffv_e85 = ce[(ce['vehicle_type']== 'FFV') & (ce['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

In [10]:
n = variable_n()
fc = variable_fc()
tac = variable_tac()
oc = variable_oc()
ce = variable_ce()


In [11]:
n1 = n.merge(B)
n2 = n1.merge(CG)
n3 = n2.merge(oc)
n4 = n3.merge(ce)
n5 = n4.merge(fc)
n6 = n5.merge(tac)
n6 = n6.sort_values(by=['county','vehicle_type'], ascending=True)
n6.reset_index(drop=True, inplace=True)
n6

,county,vehicle_type,fuel_type,count,state,fuel_cost_per_gal,operating_cost,emission_per_year,fuel_consumption,total_annual_vehicle_cost
0,Aitkin,BEV,Electricity,0,MN,2.484330,0,0,0,0
1,Aitkin,FFV,E85,0,MN,1.320000,0,0,0,0
2,Aitkin,SIDI_ICE,E10,0,MN,1.750000,0,0,0,0
3,Alameda,BEV,Electricity,0,CA,2.340349,0,0,0,0
4,Alameda,FFV,E85,0,CA,1.960000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1192,Zapata,FFV,E85,0,TX,1.110000,0,0,0,0
1193,Zapata,SIDI_ICE,E10,0,TX,1.310000,0,0,0,0
1194,Zavala,BEV,Electricity,0,TX,1.685889,0,0,0,0
1195,Zavala,FFV,E85,0,TX,1.110000,0,0,0,0


In [12]:
N_B = B.merge(N)
number_one = N_B.merge(TM, left_on=['household_income_ID','state'],right_on=['household_income_ID','state'])
number_one.drop(['household_income','household_income_ID'],axis=1,inplace=True)
n6 = n6.merge(number_one)
n6

,county,vehicle_type,fuel_type,count,state,fuel_cost_per_gal,operating_cost,emission_per_year,fuel_consumption,total_annual_vehicle_cost,median_household_income_2018,annual_miles_driven
0,Aitkin,BEV,Electricity,0,MN,2.484330,0,0,0,0,"$44,016",15583.95
1,Aitkin,FFV,E85,0,MN,1.320000,0,0,0,0,"$44,016",15583.95
2,Aitkin,SIDI_ICE,E10,0,MN,1.750000,0,0,0,0,"$44,016",11390.58
3,Alameda,BEV,Electricity,0,CA,2.340349,0,0,0,0,"$101,744",12212.65
4,Alameda,FFV,E85,0,CA,1.960000,0,0,0,0,"$101,744",12212.65
...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Zapata,FFV,E85,0,TX,1.110000,0,0,0,0,"$33,160",19491.96
1193,Zapata,SIDI_ICE,E10,0,TX,1.310000,0,0,0,0,"$33,160",10165.34
1194,Zavala,BEV,Electricity,0,TX,1.685889,0,0,0,0,"$30,076",19491.96
1195,Zavala,FFV,E85,0,TX,1.110000,0,0,0,0,"$30,076",19491.96


#### Objective Function

In [31]:
def objective():
    '''
    Minimize the total cost per mile driven in all of US (which includes the capital cost and operating cost)
    
    capital fuel cost + ((cost of fuel per gallon of vehicle)(fuel consumption of vehicle/per mile))*(total miles traveled)
    Sum this over counties and then over states
    '''    
    result = []
    for i in tqdm(range(n6.shape[0])):
        result.append((n6['count'][i] * (n6['fuel_consumption'][i] * n6['fuel_cost_per_gal'][i] * n6['annual_miles_driven'][i])))
    return list(np.add.reduceat(result, np.arange(0, len(result), 3)))
# for each county
# for each vehicle type
# compute capital cost + ((cost of fuel/gallon) * fuel consumption 1/mpg * miles_driven)
# so this comes out to CC + (CG * CF * miles_driven)
# cost vector = vehicle type * (CC +(CG * CF * miles_driven))



In [92]:
n6

,county,vehicle_type,fuel_type,count,state,fuel_cost_per_gal,operating_cost,emission_per_year,fuel_consumption,total_annual_vehicle_cost,median_household_income_2018,annual_miles_driven
0,Aitkin,BEV,Electricity,0,MN,2.484330,0,0,0,0,"$44,016",15583.95
1,Aitkin,FFV,E85,0,MN,1.320000,0,0,0,0,"$44,016",15583.95
2,Aitkin,SIDI_ICE,E10,0,MN,1.750000,0,0,0,0,"$44,016",11390.58
3,Alameda,BEV,Electricity,0,CA,2.340349,0,0,0,0,"$101,744",12212.65
4,Alameda,FFV,E85,0,CA,1.960000,0,0,0,0,"$101,744",12212.65
...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Zapata,FFV,E85,0,TX,1.110000,0,0,0,0,"$33,160",19491.96
1193,Zapata,SIDI_ICE,E10,0,TX,1.310000,0,0,0,0,"$33,160",10165.34
1194,Zavala,BEV,Electricity,0,TX,1.685889,0,0,0,0,"$30,076",19491.96
1195,Zavala,FFV,E85,0,TX,1.110000,0,0,0,0,"$30,076",19491.96


In [138]:
EF_param = {k: f.groupby('fuel_type')['emission_factor'].apply(list).to_dict()
     for k, f in EF.groupby('state')}
FE_param = {k: f.groupby('fuel_type')['fuel_economy'].apply(list).to_dict()
     for k, f in FE.groupby('vehicle_type')}
C_param = {k: f.groupby('fuel_type')['fuel_cost_per_mile'].apply(list).to_dict()
     for k, f in C.groupby('state')}
CC_param = {k: f.groupby('vehicle_type')['cost_minus_rebate'].apply(list).to_dict()
     for k, f in CC.groupby('state')}
CG_param = {k: f.groupby('fuel_type')['fuel_cost_per_gal'].apply(list).to_dict()
     for k, f in CG.groupby('state')}
D_param = 0.25
#come back to W, kind of confusing but in a fun way or do it manually
#come back to TM, seems incomplete - or may need to use n6
CF_param = {k: f.groupby('fuel_type')['fuel_economy'].apply(list).to_dict()
     for k, f in CF.groupby('vehicle_type')}
CF_param

{'BEV': {'Electricity': [0.009167863554173324]},
 'FFV': {'E85': [0.024691358024691357]},
 'SIDI_ICE': {'E10': [0.02498750624687656]}}

In [122]:
#decision variables
#n (v, f, r) 
#Projected number of vehicles of vehicle type v using fuel type f that should be in county r
n = LpVariable.dicts('number_of_vehicles', [(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')

In [123]:
#fc(r,v,f) 
#Total fuel consumption by vehicle v using fuel type f in county r
fc = LpVariable.dicts('total_fuel_use',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')

In [131]:
#oc(r,v,f) 
#Operating cost per mile for vehicle v using fuel type f in county r 
oc = LpVariable.dicts('operating_cost_permile',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')

In [134]:
#tac(r,v,f) 
#Total annual cost of vehicle v using fuel type f in county r 
tac = LpVariable.dicts('total_annual_cost',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')

In [135]:
#ce(r,v,f)  
#GHG emission per year of vehicle v using fuel type f in county r
ce = LpVariable.dicts('GHG_emissions_new',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = "Integer")

In [177]:
#OBJECTIVE FUNCTION: minimise cost, determine car allocation (mixed integer)
# prob += lpSum(CC_param[r][f] * n[(r,f)] for r in COUNTIES for f in VEHICLE_TYPES)

#SET PROBLEM VARIABLE
prob = LpProblem('Biofuels',LpMinimize)

for r in COUNTIES:
    prob += lpSum(CC_param[r][f] * n[(r,f)] for f in VEHICLE_TYPES)

TypeError: unsupported operand type(s) for +=: 'int' and 'list'

In [143]:
# for each county
# for each vehicle type
# compute capital cost + ((cost of fuel/gallon) * fuel consumption 1/mpg * miles_driven)
# so this comes out to CC + (CG * CF * miles_driven)
# cost vector = vehicle type * (CC +(CG * CF * miles_driven))
CC_param = {'BEV': [48000], 'FFV': [36000], 'SIDI_ICE': [36000]}

In [161]:
# for r in COUNTIES:
#     for f in VEHICLE_TYPES:
#         print(n[(r,f)])
CC_param = {}

{'BEV': [48000], 'FFV': [36000], 'SIDI_ICE': [36000]}

In [167]:
#next steps, develop a simple improvement on this:
#create the CC_param that is county based.

improve = CC.merge(B)
CC_param = {k: f.groupby('vehicle_type')['cost_minus_rebate'].apply(list).to_dict()
     for k, f in improve.groupby('county')}


In [178]:
for r in COUNTIES:
    for f in VEHICLE_TYPES:
#         print(CC_param[r][f])
#         print(n[(r,f)])

SyntaxError: unexpected EOF while parsing (<ipython-input-178-3c53df153d01>, line 4)

In [181]:
# CC_param[j]